# Data preparation

The purpose of this notebook is to download data (already imported and persisted in the Azure Blob Storage) and prepare it. As a result we will get a dataset ready for further analyses and modeling.

The data preparation steps are:
* [Environment configuration](#Environment-configuration)
* [Unit tests execution](#Unit-tests-execution)
* [Data ingestion, cleaning and featurization pipeline](#Data-ingestion,-cleaning-and-featurization-pipeline)
* [Quick verification of pipeline outputs](#Post-execution-verification)

## Environment configuration

In [1]:
import os
import pandas as pd

## Unit tests execution

This step is to make sure the python code (responsible for data ingestion, cleaning and featurization pipeline) is in the stable state.

In [2]:
# Let the 'bikerentals' folder be a current directory
%cd ..

/Users/mariuszrokita/GitHub/wroclawski-rower-miejski/bikerentals


In [3]:
# execute tests to make sure everything is working as expected
!python -m pytest

============================= test session starts ==============================
platform darwin -- Python 3.6.9, pytest-5.2.2, py-1.8.0, pluggy-0.13.0
rootdir: /Users/mariuszrokita/GitHub/wroclawski-rower-miejski, inifile: tox.ini
plugins: cov-2.8.1
collected 24 items                                                             

src/tests/cleaning/test_extract_gps_from_station_name.py ..              [  8%]
src/tests/cleaning/test_pipeline.py ....                                 [ 25%]
src/tests/cleaning/test_remove_missing_gps.py ..                         [ 33%]
src/tests/cleaning/test_remove_same_location.py ...                      [ 45%]
src/tests/features/test_day_of_week.py ..                                [ 54%]
src/tests/features/test_distance.py ..                                   [ 62%]
src/tests/features/test_holidays.py ...                                  [ 75%]
src/tests/features/test_hour.py ..                                       [ 83%]
src/tests/features/test_mont

## Data preparation pipeline

This step has been implemented as a full pipeline and consists of following steps:
* data ingestion (downloading already imported data from Azure Blob Storage to local destination),
* cleaning and soft/hard removing records
* creating new features
* saving dataset in the `data/processed` location

In fact, we'll execute the pipeline twice. This way we'll get two output datasets: one with soft deleted records, other one - with hard removed records.

In [4]:
# Go to source folder
%cd src

/Users/mariuszrokita/GitHub/wroclawski-rower-miejski/bikerentals/src


In [5]:
# run data loading and processing pipeline 
# (with soft deleting so that we can inspect everything)
%run -t run_pipeline.py --hard-delete=False --save=bike_rentals_soft

Script execution started
Root folder set to: /Users/mariuszrokita/GitHub/wroclawski-rower-miejski/bikerentals
Pipeline execution about to start!
**** DataPreparationPipeline stage - start ****
Input data shape: (0, 0)
    **** DataIngestion stage - start ****
    Input data shape: (0, 0)
Client-Request-ID=7554050c-0c23-11ea-bc42-4c327596358d Outgoing request: Method=GET, Path=/bike-rentals, Query={'restype': 'container', 'comp': 'list', 'prefix': None, 'delimiter': None, 'marker': None, 'maxresults': None, 'include': None, 'timeout': None}, Headers={'x-ms-version': '2018-03-28', 'User-Agent': 'Azure-Storage/1.4.2-1.5.0 (Python CPython 3.6.9; Darwin 19.0.0)', 'x-ms-client-request-id': '7554050c-0c23-11ea-bc42-4c327596358d', 'x-ms-date': 'Thu, 21 Nov 2019 05:55:02 GMT', 'Authorization': 'REDACTED'}.
Client-Request-ID=7554050c-0c23-11ea-bc42-4c327596358d Receiving Response: Server-Timestamp=Thu, 21 Nov 2019 05:55:01 GMT, Server-Request-ID=6f25a755-b01e-00d4-6730-a0fc2a000000, HTTP Status 


IPython CPU timings (estimated):
  User   :      67.01 s.
  System :       2.19 s.
Wall time:      70.82 s.


In [6]:
# run data loading and processing pipeline (with hard deleting)
%run -t run_pipeline.py --hard-delete=True --save=bike_rentals

Script execution started
Root folder set to: /Users/mariuszrokita/GitHub/wroclawski-rower-miejski/bikerentals
Pipeline execution about to start!
**** DataPreparationPipeline stage - start ****
Input data shape: (0, 0)
    **** DataIngestion stage - start ****
    Input data shape: (0, 0)
Client-Request-ID=9f34a138-0c23-11ea-ab71-4c327596358d Outgoing request: Method=GET, Path=/bike-rentals, Query={'restype': 'container', 'comp': 'list', 'prefix': None, 'delimiter': None, 'marker': None, 'maxresults': None, 'include': None, 'timeout': None}, Headers={'x-ms-version': '2018-03-28', 'User-Agent': 'Azure-Storage/1.4.2-1.5.0 (Python CPython 3.6.9; Darwin 19.0.0)', 'x-ms-client-request-id': '9f34a138-0c23-11ea-ab71-4c327596358d', 'x-ms-date': 'Thu, 21 Nov 2019 05:56:12 GMT', 'Authorization': 'REDACTED'}.
Client-Request-ID=9f34a138-0c23-11ea-ab71-4c327596358d Receiving Response: Server-Timestamp=Thu, 21 Nov 2019 05:56:12 GMT, Server-Request-ID=7af23913-f01e-0043-1230-a0aa27000000, HTTP Status 


IPython CPU timings (estimated):
  User   :      57.90 s.
  System :       1.95 s.
Wall time:      61.09 s.


## Post-execution verification

In [7]:
# set up paths to created datasets
filepath_soft = os.path.join(os.getcwd(), '..', 'data', 'processed', 'bike_rentals_soft.csv')
filepath_hard = os.path.join(os.getcwd(), '..', 'data', 'processed', 'bike_rentals.csv')

In [8]:
# load data
bike_rentals_soft_df = pd.read_csv(filepath_soft)
bike_rentals_hard_df = pd.read_csv(filepath_hard)

In [9]:
print('Dataset with soft deleted records: ', bike_rentals_soft_df.shape)
print('Dataset with hard deleted records: ', bike_rentals_hard_df.shape)

Dataset with soft deleted records:  (480640, 17)
Dataset with hard deleted records:  (345999, 16)
